## SILVER TO GOLD LAYER

### Gold Layer - Trade History


In [11]:
## Import necessary libraries and utility functions
import pandas as pd

from PortfolioTracker.GlobalPath import GlobalPath
from PortfolioTracker.Portfolio import Portfolio

In [12]:
# Instantiate GlobalPath
global_path = GlobalPath("PortfolioTracker")
# GLOBAL PATH
tradehistory_silver_file_path = global_path.joinpath(
    "DATA/SILVER/TradeHistory/TradeHistory_data.csv"
)
profitloss_gold_file_path = global_path.joinpath(
    "DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"
)
stockprice_silver_file_path = global_path.joinpath(
    "DATA/SILVER/StockPrice/StockPrice_data.csv"
)
holdings_gold_file_path = global_path.joinpath(
    "DATA/GOLD/Holdings/Holdings_data.csv"
)

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [13]:
# Read the CSV file
df_trade_history = pd.read_csv(tradehistory_silver_file_path)

# Convert 'datetime' to datetime type
df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

# Sort the DataFrame by 'datetime'
df_trade_history = df_trade_history.sort_values(by="datetime")

print(
    f"Read SILVER Layer trade history data from: {tradehistory_silver_file_path}"
)

Read SILVER Layer trade history data from: ..\..\DATA\SILVER\TradeHistory\TradeHistory_data.csv


### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks


In [14]:
# Apply the trade logic to each row of the DataFrame
# Instantiate the Portfolio object
portfolio = Portfolio()
for record in df_trade_history.astype(str).to_dict(orient="records"):
    portfolio.trade(record)

portfolio.check_expired_stocks()
# expired_stocks example : df_trade_history["stock_name"] == "NIFTY-PE-24650-18JUL2024"

NIFTY-PE-24650-18JUL2024 => 100.0 expired
NIFTY-CE-25000-08AUG2024 => 25.0 expired


### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [15]:
# Create a DataFrame from the processed data
df_pnl = pd.DataFrame(portfolio.get_pnl())

# Sort the DataFrame
df_pnl = df_pnl.sort_values(
    by=[
        "segment",
        "exchange",
        "symbol",
        "stock_name",
        "close_datetime",
        "open_datetime",
    ]
)

# Round the values in to two decimal places
df_pnl = df_pnl.round(2)

# Reset index to ensure it starts from 0
df_pnl = df_pnl.reset_index(drop=True)

# Select and reorder the columns for the final DataFrame
df_pnl = df_pnl[
    [
        "exchange",
        "segment",
        "symbol",
        "stock_name",
        "position",
        "quantity",
        "open_datetime",
        "open_side",
        "open_price",
        "open_amount",
        "close_datetime",
        "close_side",
        "close_price",
        "close_amount",
        "pnl_amount",
        "pnl_percentage",
    ]
]

# Save the final DataFrame to a CSV file
df_pnl.to_csv(profitloss_gold_file_path, index=None)

print("GOLD Layer CSV file for ProfitLoss successfully created at:")
print(profitloss_gold_file_path)

# Display the DataFrame debugrmation
df_pnl.info()

GOLD Layer CSV file for ProfitLoss successfully created at:
..\..\DATA\GOLD\ProfitLoss\ProfitLoss_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   exchange        133 non-null    object        
 1   segment         133 non-null    object        
 2   symbol          133 non-null    object        
 3   stock_name      133 non-null    object        
 4   position        133 non-null    object        
 5   quantity        133 non-null    float64       
 6   open_datetime   133 non-null    datetime64[ns]
 7   open_side       133 non-null    object        
 8   open_price      133 non-null    float64       
 9   open_amount     133 non-null    float64       
 10  close_datetime  133 non-null    datetime64[ns]
 11  close_side      133 non-null    object        
 12  close_price     133 non-null    float64       
 13  clo

In [16]:
# Create a DataFrame from the processed data
df_holdings = pd.DataFrame(portfolio.get_holdings())

# Filter the DataFrame to include only rows where the segment is 'EQ' (Equity) or 'MF' (Mutual Funds)
df_holdings = df_holdings[df_holdings["segment"].isin(["EQ", "MF"])]

# Round the values in the columns to two decimal places
df_holdings = df_holdings.round(2)

# Extract date from datetime
df_holdings["date"] = df_holdings["datetime"].dt.date

# Find the index of the maximum datetime for each stock_name and date
idx = df_holdings.groupby(["stock_name", "date"])["datetime"].idxmax()

# Use the indices to filter the original DataFrame
df_holdings = df_holdings.loc[idx].reset_index(drop=True)

In [17]:
# Create a function to expand the date range for each stock


def expand_dates(stock_df):
    """
    Expands the date range for each stock to include all dates from the
    minimum date to the current date and forward fills the missing values.

    Parameters:
    stock_df (pd.DataFrame): DataFrame containing stock data for a single stock.

    Returns:
    pd.DataFrame: DataFrame with expanded date range and forward-filled values.
    """
    # Find the minimum date for this stock
    min_date = stock_df["date"].min()

    # Create a date range from the minimum date to today
    date_range = pd.date_range(start=min_date, end=pd.to_datetime("today"))

    # Reindex the stock_df to include the full date range
    stock_df = stock_df.set_index("date").reindex(date_range)

    # Forward fill the values to fill missing dates
    stock_df = stock_df.ffill().reset_index()

    # Rename the 'index' column to 'date'
    stock_df = stock_df.rename(columns={"index": "date"})

    return stock_df


# Apply the expand_dates function to each stock group
df_holdings = (
    df_holdings.groupby("stock_name")
    .apply(expand_dates, include_groups=False)
    .reset_index()
)

In [18]:
# Load stock prices data from the specified CSV file path
df_StockPrice = pd.read_csv(stockprice_silver_file_path)

# Convert the 'date' column in stock prices to datetime type for consistency
df_StockPrice["date"] = pd.to_datetime(df_StockPrice["date"])

print(
    f"Loaded SILVER Layer stock price data from: {stockprice_silver_file_path}"
)

# Merge the expanded holdings data with the stock price data
df_holdings = pd.merge(
    df_holdings,
    df_StockPrice,
    on=["date", "stock_name"],
    how="left",
)

Loaded SILVER Layer stock price data from: ..\..\DATA\SILVER\StockPrice\StockPrice_data.csv


In [19]:
# Calculate the OHLC value of holdings
col_names = ["open", "high", "low", "close"]
for col_name in col_names:
    df_holdings[f"{col_name}_price"] = df_holdings[col_name]
    df_holdings[f"{col_name}_amount"] = (
        df_holdings[col_name] * df_holdings["holding_quantity"]
    )

# Forward fill the values to fill missing dates
df_holdings = df_holdings.ffill()

# Filter out rows with zero holding quantity
df_holdings = df_holdings[df_holdings["holding_quantity"] != 0]

# Round the values in to two decimal places
df_holdings = df_holdings.round(2)

In [20]:
# Sort the DataFrame for organized output
df_holdings = df_holdings.sort_values(
    by=[
        "date",
        "segment",
        "exchange",
        "symbol",
        "stock_name",
    ]
)

# Reset index to ensure it starts from 0
df_holdings = df_holdings.reset_index(drop=True)

# Select and reorder the columns for the final DataFrame
df_holdings = df_holdings[
    [
        "date",
        "segment",
        "exchange",
        "symbol",
        "stock_name",
        "holding_quantity",
        "avg_price",
        "holding_amount",
        "open_price",
        "open_amount",
        "high_price",
        "high_amount",
        "low_price",
        "low_amount",
        "close_price",
        "close_amount",
    ]
]

# Save the final processed DataFrame to a new CSV file
df_holdings.to_csv(holdings_gold_file_path, index=None)

print(
    f"GOLD Layer CSV file for Holdings successfully created at: {holdings_gold_file_path}"
)

# Display debugrmation about the final DataFrame
df_holdings.info()

GOLD Layer CSV file for Holdings successfully created at: ..\..\DATA\GOLD\Holdings\Holdings_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5977 entries, 0 to 5976
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              5977 non-null   datetime64[ns]
 1   segment           5977 non-null   object        
 2   exchange          5977 non-null   object        
 3   symbol            5977 non-null   object        
 4   stock_name        5977 non-null   object        
 5   holding_quantity  5977 non-null   float64       
 6   avg_price         5977 non-null   float64       
 7   holding_amount    5977 non-null   float64       
 8   open_price        5977 non-null   float64       
 9   open_amount       5977 non-null   float64       
 10  high_price        5977 non-null   float64       
 11  high_amount       5977 non-null   float64       
 12  low_price         5977 non-null